## Dock Dock MobyDock

Cleanup: Delete all images on the host  
Stop and delete all the containers being used by images.  
`docker rmi $(docker images -aq)`  

Move to the directory first by using the cd command and verify the path of the working directory from pwd command?  
`cd /root/webapp-color/`  
`docker build -t web app-color .`  #At the end of the command, we used the "." (dot) symbol which indicates for the current directory, so you need to run this command from within the directory that has the Dockerfile.

Which operating system is used as base image for python:3.6  
`docker run python:3.6 cat /etc/*release*`  

Create a docker-compose.yml file under the directory /root/clickcounter. Once done, run docker-compose up.  
The compose file should have the exact specification as follows -  
	1	redis service specification - Image name should be redis:alpine.  
	2	clickcounter service specification - Image name should be kodekloud/click-counter, app is run on port 5000 and expose it on the host port 8085 in the compose file.  
`services:`  
..`redis:`  
....`image: redis:alpine`  
..`clickcounter:`  
....`image: kodekloud/click-counter`  
....`ports:`  
....`- 8085:5000`  
`version: '3.0'`

`docker-compose up -d`

Let practice deploying a registry server on our own.  
Run a registry server with name equals to my-registry using registry:2 image with host port set to 5000, and restart policy set to always.  
Note: Registry server is exposed on port 5000 in the image.  
`docker run -d -p 5000:5000 --restart=always --name my-registry registry:2`

Now its time to push some images to our registry server. Let's push two images for now .i.e. nginx:latest and httpd:latest.
Note: Don't forget to pull them first.  
To check the list of images pushed , use curl -X GET localhost:5000/v2/_catalog    
Run:   
`docker pull nginx:latest`   
`docker image tag nginx:latest localhost:5000/nginx:latest`   
and finally push it using     
`docker push localhost:5000/nginx:latest`  
We will use the same steps for the second image   
`docker pull httpd:latest` and then   
`docker image tag httpd:latest localhost:5000/httpd:latest`   
and finally push it using `docker push localhost:5000/httpd:latest`  

To remove images - `docker image prune -a`

Now we can pull images from our registry-server as well. Use `docker pull [server-addr/image-name]` to pull the images that we pushed earlier.  
In our case we can use: `docker pull localhost:5000/nginx`  


Number of networks- `docker network ls`  

We just ran a container named alpine-1. Identify the network it is attached to. `docker inspect alpine-1`

What is the subnet configured on bridge network?  `docker inspect bridge`  

Run a container named alpine-2 using the alpine image and attach it to the none network. `docker run -d --name=alpine-2 --network=none alpine`

Create a new network named wp-mysql-network using the bridge driver. Allocate subnet 182.18.0.1/24. Configure Gateway 182.18.0.1   `docker network create --driver=bridge --subnet=182.18.0.1/24 --gateway=182.18.0.1 wp-mysql-network`

Deploy a mysql database using the mysql:5.6 image and name it mysql-db. Attach it to the newly created network wp-mysql-network
Set the database password to use db_pass123. The environment variable to set is MYSQL_ROOT_PASSWORD.  
`docker run -d --name=mysql-db --network=wp-mysql-network -e MYSQL_ROOT_PASSWORD=db_pass123 mysql:5.6`

Deploy a web application named webapp using the kodekloud/simple-webapp-mysql image. Expose the port to 38080 on the host.The application makes use of two environment variable:  
1: DB_Host with the value mysql-db.  
2: DB_Password with the value db_pass123.  
Make sure to attach it to the newly created network called wp-mysql-network.  
Also make sure to link the MySQL and the webapp container.  
`docker run --network=wp-mysql-network -e DB_Host=mysql-db -e DB_Password=db_pass123 -p 38080:8080 --name webapp --link mysql-db:mysql-db -d kodekloud/simple-webapp-mysql`

